*On* importe tous les bibliothèques nécessaires pour créer les modèles Glove, FastText et Word2vec et aussi les bibliothèques qui peuvent nous aider à trouver la ressemblance entre les mots pour les trois modèles




In [ ]:
! pip install glove_python
! pip install arabic_reshaper
! pip install python-bidi
import tensorflow as tf
from keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec, keyedvectors,FastText
from glove import Corpus, Glove
import numpy as np
from bs4 import BeautifulSoup
import re
import arabic_reshaper
from bidi.algorithm import get_display

puis on ouvre le fichier text CorpusA et on lit chaque ligne du fichier ensuite on stocke ces lignes dans une liste L1, Lorsqu' on affiche les lignes de la liste un caractère s'affiche ,ce dernier doit etre supprimé. 

In [ ]:
def clean(text):
        # suppression des carac html
        example1 = BeautifulSoup(text, 'lxml')

        # suppression des mentions
        example2 = re.sub(r'@[A-Za-z0-9]+', '', example1.get_text())

        # suppression des liens
        ex3 = re.sub('https?://[A-Za-z0-9./]+', '', example2)

        # suppression des hashtag
        final = re.sub(r'#[A-Za-z0-9]+', "", ex3)

        def remove_emojis(data):
            emoj = re.compile("["
                        u"\U0001f600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002500-\U00002BEF"  # chinese char
                        u"\U00002702-\U000027B0"
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        u"\U0001f926-\U0001f937"
                        u"\U00010000-\U0010ffff"
                        u"\u2640-\u2642"
                        u"\u2600-\u2B55"
                        u"\u200d"
                        u"\u23cf"
                        u"\u23e9"
                        u"\u231a"
                        u"\ufe0f"  # dingbats
                        u"\u3030"
                        "]+", re.UNICODE)
           return re.sub(emoj, '', data) 
           final=remove_emojis(finale)
        return final

In [ ]:
f = open('CorpusA.txt', mode = 'r', encoding='utf-8')
L1 = f.readlines()
help = L1[0].replace('\ufeff','')
del L1[0]
L1.insert(0,help)
data=L1
L1=[]
for l1 in data :
    L1.append(clean(l1))

Lors de cette etape ,on divisé les phrases en mots .

In [ ]:
L2=[text_to_word_sequence(q) for q in L1]

Dans cette etape on creée notre modele fasttext et word2vect

In [ ]:
ft_model = Word2Vec(sentences=L2, size=1000, window=5, sample=1e-2, negative=10, sg=0, workers=10, min_count=1,iter=20, compute_loss=True)
ft_modelF = FastText(sentences=L2, size=1000, window=5, sample=1e-2, negative=10, sg=0, workers=10, min_count=1,iter=20)

Puis arrive le calcul de la similarité de 2 mots suivant les 2 modeles

In [ ]:
print(ft_model.wv.similarity(w1='الفستان', w2='القميص'))
print(ft_model.wv.similarity(w1='البنطال', w2='القميص'))
print(ft_model.wv.similarity(w1='الأسود', w2='الأزرق'))
print(ft_model.wv.similarity(w1='الأحمر', w2='الأزرق'))
print(ft_model.wv.similarity(w1='الأبيض', w2='الأزرق'))
print(ft_model.wv.similarity(w1='الخمري', w2='الأزرق'))
print('\n')
print('\n')
dst1 = (np.dot(ft_model.wv['الفستان'], ft_model.wv['القميص'])
       / np.linalg.norm(ft_model.wv['الفستان'])
       / np.linalg.norm(ft_model.wv['القميص']))
print(dst1)
print('\n')
print('\n')
print(ft_modelF.wv.similarity(w1='الفستان', w2='القميص'))
print(ft_modelF.wv.similarity(w1='البنطال', w2='القميص'))
print(ft_modelF.wv.similarity(w1='الأسود', w2='الأزرق'))
print(ft_modelF.wv.similarity(w1='الأحمر', w2='الأزرق'))
print(ft_modelF.wv.similarity(w1='الأبيض', w2='الأزرق'))
print(ft_modelF.wv.similarity(w1='الخمري', w2='الأزرق'))

0.25283965
0.559656
0.6484264
0.6082984
0.5814122
0.6410667




0.2528396




0.21616165
0.51945144
0.76615363
0.75682425
0.73480904
0.7200477


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Maintenant on cree notre modele glove

In [ ]:
corpus = Corpus() 

corpus.fit(L2, window=10)

glove = Glove(no_components=100, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=10, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 30 training epochs with 10 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [ ]:
#print(glove.word_vectors[glove.dictionary['القميص']])

print(glove.most_similar('القميص',11))


[('الزهري', 0.8601455906930572), ('الأبيض', 0.8462820995174442), ('النسائي', 0.8110687943555177), ('الأصفر', 0.8089756275175853), ('الأحمر', 0.7875086947773243), ('السماوي', 0.7817391824728276), ('قمصان', 0.7693305558213159), ('قميص', 0.7568918997153601), ('بلوز', 0.7483318827388953), ('الفستان', 0.7451595280057982)]


In [ ]:
dst = (np.dot(glove.word_vectors[glove.dictionary['القميص']], glove.word_vectors[glove.dictionary['الفستان']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['القميص']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['الفستان']]))

In [ ]:
print(dst)

0.7451595280057982


Suite a l abscence de une fonction predefinie afin de calculer la similarité on creer un petit programme qui nous realisera cette tache .

In [ ]:
dst1 = (np.dot(glove.word_vectors[glove.dictionary['الفستان']], glove.word_vectors[glove.dictionary['القميص']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['الفستان']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['القميص']]))
dst2 = (np.dot(glove.word_vectors[glove.dictionary['البنطال']], glove.word_vectors[glove.dictionary['القميص']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['البنطال']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['القميص']]))
dst3 = (np.dot(glove.word_vectors[glove.dictionary['الأسود']], glove.word_vectors[glove.dictionary['الأزرق']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['الأسود']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['الأزرق']]))
dst4 = (np.dot(glove.word_vectors[glove.dictionary['الأحمر']], glove.word_vectors[glove.dictionary['الأزرق']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['الأحمر']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['الأزرق']]))
dst5 = (np.dot(glove.word_vectors[glove.dictionary['الأبيض']], glove.word_vectors[glove.dictionary['الأزرق']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['الأبيض']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['الأزرق']]))
dst6 = (np.dot(glove.word_vectors[glove.dictionary['الخمري']], glove.word_vectors[glove.dictionary['الأزرق']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['الخمري']])
       / np.linalg.norm(glove.word_vectors[glove.dictionary['الأزرق']]))
print(dst1)
print(dst2)
print(dst3)
print(dst4)
print(dst5)
print(dst6)

0.7451595280057981
0.5807796041158897
0.5842232981296477
0.6202342184392456
0.6902274056358961
0.5205963659659341
